# V.A. Memory networks

In a standard network the flow has no memory, which means that a random walker moves to any neighbor with a probability proportional to the link weights, independent of where it came from in previous steps.

For some type of data, this limitation is hides important patterns. For example, if we model flight patterns where people move between airports, it is common for people to return to the previously visited airport. This behaviour can be modelled by adding memory to the random walker as a proxy for flow.

In [6]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import infomap
import json
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import tempfile
from pathlib import Path


In [2]:
%reload_ext autoreload
%autoreload 2
import util

## Model memory with state networks

A general way to represent memory is by using a _state network_, where each physical node contains state nodes representing memory of some sort, for example the last visited physical node.

<img src="https://www.mapequation.org/infomap/images/state-network.svg" alt="state-network" width="400"/>

Here we have five physical nodes $i \ldots m$. In the middle node, we have two state nodes that keeps the left and right triangular flow relatively separate; the left state node redirect most of the flow back to the left physical nodes and vice versa.

Infomap partition the network on the state level, with the left and right state nodes in two different modules. The middle physical node is then part of two different modules, which means that the modules can overlap on the physical level.



In [9]:
network = """
# The state network above
*Vertices 5
#node_id name
1 "i" 
2 "j" 
3 "k" 
4 "l" 
5 "m" 
*States
#state_id node_id name
1 1 "α~_i" 
2 2 "β~_j" 
3 3 "γ~_k" 
4 1 "δ~_i" 
5 4 "ε~_l" 
6 5 "ζ~_m" 
*Links
#source target weight
1 2 0.8 
1 3 0.8 
1 5 0.2 
1 6 0.2 
2 1 1 
2 3 1 
3 1 1 
3 2 1 
4 5 0.8 
4 6 0.8 
4 2 0.2 
4 3 0.2 
5 4 1 
5 6 1 
6 4 1 
6 5 1 
"""
with open("data/state_network.net", "w") as fp:
    fp.write(network)

In [21]:
im = infomap.Infomap(silent=True)
im.read_file("data/state_network.net")
im.run()
print(f"Found {im.num_top_modules} modules with codelength {im.codelength:.8f} bits")
im.get_dataframe(["node_id", "name", "state_id", "module_id", "flow"]).set_index("state_id")

Found 2 modules with codelength 2.01140524 bits


,node_id,name,module_id,flow
state_id,,,,
1,1,i,1,0.166667
2,2,j,1,0.166667
3,3,k,1,0.166667
4,1,i,2,0.166667
5,4,l,2,0.166667
6,5,m,2,0.166667


In the above table we see that the network is partitioned in two modules overlapping on physical node _i_.

Below is an example to programmatically add a state network using the Infomap api

In [20]:
im = infomap.Infomap(two_level=True, silent=True)

# Set names on physical nodes
im.set_name(1, "PRE")
im.set_name(2, "SCIENCE")
im.set_name(3, "PRL")
im.set_name(4, "BIO")

# First number is state id, second is node id (physical node)
im.add_state_node(0, 1)
im.add_state_node(1, 2)
im.add_state_node(2, 3)
im.add_state_node(3, 2)
im.add_state_node(4, 2)
im.add_state_node(5, 4)

# Links connect state nodes
im.add_link(0, 1)
im.add_link(1, 2)
im.add_link(3, 2)
im.add_link(4, 5)

im.run()

print(f"Found {im.num_top_modules} modules with codelength {im.codelength:.8f} bits")

im.get_dataframe(["node_id", "name", "state_id", "module_id", "flow"]).set_index("state_id")

Found 2 modules with codelength 1.34436094 bits


,node_id,name,module_id,flow
state_id,,,,
1,2,SCIENCE,1,0.250
2,3,PRL,1,0.250
0,1,PRE,1,0.125
3,2,SCIENCE,1,0.125
4,2,SCIENCE,2,0.125
5,4,BIO,2,0.125
